In [24]:
#testing results with Lasso and Elastic Net - expected 15/260 most important features
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, ElasticNet

In [17]:
all_data = pd.read_csv('../data/election_census_final')
description = pd.read_csv('../data/var_description.csv')

In [20]:
y = all_data.target

In [21]:
X = all_data.iloc[:, 8:268]

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

In [40]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit(X_test)
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=0)
elastic_net.fit(X_train_scaled, y_train).coef_

array([-0.        , -0.        , -0.        , -0.        ,  0.        ,
        0.        ,  0.        , -0.        , -0.        , -0.        ,
        0.        , -0.        ,  0.        ,  0.        ,  0.        ,
       -0.        ,  0.        ,  0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        ,  0.        , -0.        ,
       -0.        , -0.        , -0.        ,  0.        ,  0.07135662,
        0.        ,  0.03555441, -0.        ,  0.        , -0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.0174592 ,
       -0.        , -0.        ,  0.        ,  0.        , -0.        ,
        0.        , -0.        , -0.        ,  0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        ,  0.        ,
       -0.        ,  0.        , -0.        , -0.        ,  0.        ,
       -0.00578201, -0.        , -0.01602839,  0.        , -0.        ,
       -0.        ,  0.        ,  0.        , -0.        , -0.  

In [39]:
#this is way we could not use Lass0 --> too many correlated features
lasso_reg = Lasso()
lasso_reg.fit(X_train_scaled, y_train).coef_

array([-0., -0., -0., -0.,  0.,  0.,  0., -0., -0., -0.,  0., -0.,  0.,
        0.,  0., -0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0., -0.,
       -0., -0.,  0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0.,
       -0., -0., -0.,  0.,  0., -0.,  0., -0., -0.,  0., -0., -0., -0.,
       -0.,  0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0., -0.,
       -0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0., -0., -0.,  0.,
       -0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  0.,  0., -0.,  0.,
        0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,
        0., -0.,  0., -0., -0.,  0., -0., -0., -0., -0., -0., -0.,  0.,
       -0.,  0.,  0., -0., -0.,  0., -0., -0., -0.,  0.,  0., -0., -0.,
       -0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  0.,  0.,
        0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0., -0., -0.,
        0., -0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0., -0., -0.,
        0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  0., -0., -0

In [39]:
#using Elastc net instead of Lasso as features are numerous and highly correlated
from sklearn.linear_model import ElasticNet
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic_net.fit(X, y)
elastic_net.predict([[1.5]])


array([0.44147156])